In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [1]:
print(tf.__version__)

NameError: name 'tf' is not defined

In [ ]:
import os
dataset_path = f"{os.getcwd()}\lung"
print(os.listdir(dataset_path))

In [ ]:
train_dir = f"{dataset_path}/Train"
val_dir = f"{dataset_path}/Val"
test_dir = f"{dataset_path}/Test"

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=123,
    image_size=(768,768),
    batch_size=32,
    shuffle=True
)

In [ ]:
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    seed=123,
    image_size=(768,768),
    batch_size=32,
    shuffle=True
)

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=123,
    image_size=(768,768),
    batch_size=32,
    shuffle=False
)

In [ ]:
# Print class names
class_names = train_dataset.class_names
print("Class Names:", class_names)

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):  # Display 9 images
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

In [ ]:
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(768, 768),
    layers.Rescaling(1.0/255),
])

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),  #Randomly flips images horizontally and vertically.
    layers.RandomRotation(0.2), #Randomly rotates the image by a factor of 20% of 360 degrees (i.e., up to 72 degrees).
])

In [ ]:
train_ds = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
n_classes = 3

model = models.Sequential([
    layers.Input(shape=(768, 768, 3)),  # Use `shape` instead of `input_shape`
    resize_and_rescale,  # Resize and rescale input images
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    batch_size=32,
    validation_data=val_dataset,
    verbose=1,
    epochs=30,
)

In [ ]:
scores = model.evaluate(test_dataset)

In [ ]:
histort_dict = history.history
histort_dict.keys()

In [ ]:
accuracy_scores = histort_dict['accuracy']
val_accuracy_scores = histort_dict['val_accuracy']

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.plot(range(len(accuracy_scores)), accuracy_scores, label='Training Accuracy')
plt.plot(range(len(val_accuracy_scores)), val_accuracy_scores, label='Validation Accuracy')
plt.xlabel('Number of Epochs')
plt.ylabel('Training Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
plt.plot(range(len(loss)), loss, label='Training Loss')
plt.plot(range(len(val_loss)), val_loss, label='Validation Loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Training Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
for images_batch, labels_batch in test_dataset.take(1): 

    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    plt.imshow(first_image)
    print("actual label:",class_names[first_label])

In [ ]:
batch_prediction = model.predict(images_batch)
predicted_label_index = np.argmax(batch_prediction[0])
predicted_label = class_names[predicted_label_index]
actual_label = class_names[first_label]

# Plot the image
plt.figure(figsize=(6, 6))
plt.imshow(first_image)

plt.title(f"Actual Label: {actual_label}\nPredicted Label: {predicted_label}", fontsize=14)
plt.axis('off')
plt.show()

In [ ]:
model.save('my_model.h5')